# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide. 

Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('omw-1.4')
from nltk.probability import FreqDist
from nltk.classify.api import ClassifierI

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
new_skip = np.linspace(1, 1599999, 1579999)

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Ironhack/Bootcamp/NLP/Lab/training.1600000.processed.noemoticon.csv', encoding="ISO-8859-1", skiprows=new_skip)
df.columns = ['target','id','date','flag','user','text']
df

,target,id,date,flag,user,text
0,0,1467834053,Mon Apr 06 22:25:52 PDT 2009,NO_QUERY,thelazyboy,sleep soon... i just hate saying bye and see y...
1,0,1467853356,Mon Apr 06 22:30:54 PDT 2009,NO_QUERY,dbmendel,Picked Mich St to win it all from the get go. ...
2,0,1467873004,Mon Apr 06 22:36:03 PDT 2009,NO_QUERY,omgitsjo,I miss him. Can't wait to celebrate the Tar H...
3,0,1467894600,Mon Apr 06 22:41:51 PDT 2009,NO_QUERY,dreaaa,throat is closing up and i had some string che...
4,0,1467913111,Mon Apr 06 22:46:57 PDT 2009,NO_QUERY,nssmom,#3 woke up and was having an accident - &quot;...
...,...,...,...,...,...,...
19995,4,2193503437,Tue Jun 16 08:32:47 PDT 2009,NO_QUERY,alethiatoh,@jasontancy thanks!
19996,4,2193527841,Tue Jun 16 08:34:46 PDT 2009,NO_QUERY,astroross,@DailyStatsmobi it was an amazing race and the...
19997,4,2193552007,Tue Jun 16 08:36:44 PDT 2009,NO_QUERY,Bellaleyla,"@Sky_1995 sorryyy. i gotta go now, my mom want..."
19998,4,2193575834,Tue Jun 16 08:38:42 PDT 2009,NO_QUERY,arikabu,@planetchuckie haha! i wonder why i made her m...


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [5]:
def clean_up(s):
    """
    Cleans up numbers, URLs, and special characters from a string.

    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """
    clean_str = s
    clean_str = clean_str.lower()
    # Replace urls by ''
    clean_str = re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', ' ' , clean_str) 
    # Replace html tags by ''
    clean_str = BeautifulSoup(clean_str, 'lxml').get_text().strip()
    clean_str = re.sub(r'\W+',' ',clean_str) # Replace everything non-alpahnumeric by ' '
    clean_str = re.sub(r'\s+',' ',clean_str) # Replace one or more whitespaces by  ' '
    clean_str = re.sub(r'\d+',' ',clean_str) # Replace one or more digits by  ' '
    clean_str = re.sub(r'<',' ',clean_str) # Replace one or more digits by  ' '
    return clean_str

In [6]:
df['text_processed'] = df['text'].apply(clean_up)
df

<ipython-input-5-ffb42e0485ec>:16: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  clean_str = BeautifulSoup(clean_str, 'lxml').get_text().strip()


,target,id,date,flag,user,text,text_processed
0,0,1467834053,Mon Apr 06 22:25:52 PDT 2009,NO_QUERY,thelazyboy,sleep soon... i just hate saying bye and see y...,sleep soon i just hate saying bye and see you ...
1,0,1467853356,Mon Apr 06 22:30:54 PDT 2009,NO_QUERY,dbmendel,Picked Mich St to win it all from the get go. ...,picked mich st to win it all from the get go w...
2,0,1467873004,Mon Apr 06 22:36:03 PDT 2009,NO_QUERY,omgitsjo,I miss him. Can't wait to celebrate the Tar H...,i miss him can t wait to celebrate the tar hee...
3,0,1467894600,Mon Apr 06 22:41:51 PDT 2009,NO_QUERY,dreaaa,throat is closing up and i had some string che...,throat is closing up and i had some string che...
4,0,1467913111,Mon Apr 06 22:46:57 PDT 2009,NO_QUERY,nssmom,#3 woke up and was having an accident - &quot;...,woke up and was having an accident it s pus...
...,...,...,...,...,...,...,...
19995,4,2193503437,Tue Jun 16 08:32:47 PDT 2009,NO_QUERY,alethiatoh,@jasontancy thanks!,jasontancy thanks
19996,4,2193527841,Tue Jun 16 08:34:46 PDT 2009,NO_QUERY,astroross,@DailyStatsmobi it was an amazing race and the...,dailystatsmobi it was an amazing race and the...
19997,4,2193552007,Tue Jun 16 08:36:44 PDT 2009,NO_QUERY,Bellaleyla,"@Sky_1995 sorryyy. i gotta go now, my mom want...",sky_ sorryyy i gotta go now my mom want to g...
19998,4,2193575834,Tue Jun 16 08:38:42 PDT 2009,NO_QUERY,arikabu,@planetchuckie haha! i wonder why i made her m...,planetchuckie haha i wonder why i made her my...


In [7]:
def tokenize(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    tokens = nltk.word_tokenize(s)
    return tokens

In [8]:
df['text_processed'] = df['text_processed'].apply(tokenize)
df

,target,id,date,flag,user,text,text_processed
0,0,1467834053,Mon Apr 06 22:25:52 PDT 2009,NO_QUERY,thelazyboy,sleep soon... i just hate saying bye and see y...,"[sleep, soon, i, just, hate, saying, bye, and,..."
1,0,1467853356,Mon Apr 06 22:30:54 PDT 2009,NO_QUERY,dbmendel,Picked Mich St to win it all from the get go. ...,"[picked, mich, st, to, win, it, all, from, the..."
2,0,1467873004,Mon Apr 06 22:36:03 PDT 2009,NO_QUERY,omgitsjo,I miss him. Can't wait to celebrate the Tar H...,"[i, miss, him, can, t, wait, to, celebrate, th..."
3,0,1467894600,Mon Apr 06 22:41:51 PDT 2009,NO_QUERY,dreaaa,throat is closing up and i had some string che...,"[throat, is, closing, up, and, i, had, some, s..."
4,0,1467913111,Mon Apr 06 22:46:57 PDT 2009,NO_QUERY,nssmom,#3 woke up and was having an accident - &quot;...,"[woke, up, and, was, having, an, accident, it,..."
...,...,...,...,...,...,...,...
19995,4,2193503437,Tue Jun 16 08:32:47 PDT 2009,NO_QUERY,alethiatoh,@jasontancy thanks!,"[jasontancy, thanks]"
19996,4,2193527841,Tue Jun 16 08:34:46 PDT 2009,NO_QUERY,astroross,@DailyStatsmobi it was an amazing race and the...,"[dailystatsmobi, it, was, an, amazing, race, a..."
19997,4,2193552007,Tue Jun 16 08:36:44 PDT 2009,NO_QUERY,Bellaleyla,"@Sky_1995 sorryyy. i gotta go now, my mom want...","[sky_, sorryyy, i, got, ta, go, now, my, mom, ..."
19998,4,2193575834,Tue Jun 16 08:38:42 PDT 2009,NO_QUERY,arikabu,@planetchuckie haha! i wonder why i made her m...,"[planetchuckie, haha, i, wonder, why, i, made,..."


In [9]:
def stem_and_lemmatize(l):
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """

    port = PorterStemmer()
    stem = [port.stem(word) for word in l]

    lem = WordNetLemmatizer()

    stem_lem = [lem.lemmatize(word) for word in stem]
    
    return stem_lem

In [10]:
df['text_processed'] = df['text_processed'].apply(stem_and_lemmatize)
df

,target,id,date,flag,user,text,text_processed
0,0,1467834053,Mon Apr 06 22:25:52 PDT 2009,NO_QUERY,thelazyboy,sleep soon... i just hate saying bye and see y...,"[sleep, soon, i, just, hate, say, bye, and, se..."
1,0,1467853356,Mon Apr 06 22:30:54 PDT 2009,NO_QUERY,dbmendel,Picked Mich St to win it all from the get go. ...,"[pick, mich, st, to, win, it, all, from, the, ..."
2,0,1467873004,Mon Apr 06 22:36:03 PDT 2009,NO_QUERY,omgitsjo,I miss him. Can't wait to celebrate the Tar H...,"[i, miss, him, can, t, wait, to, celebr, the, ..."
3,0,1467894600,Mon Apr 06 22:41:51 PDT 2009,NO_QUERY,dreaaa,throat is closing up and i had some string che...,"[throat, is, close, up, and, i, had, some, str..."
4,0,1467913111,Mon Apr 06 22:46:57 PDT 2009,NO_QUERY,nssmom,#3 woke up and was having an accident - &quot;...,"[woke, up, and, wa, have, an, accid, it, s, pu..."
...,...,...,...,...,...,...,...
19995,4,2193503437,Tue Jun 16 08:32:47 PDT 2009,NO_QUERY,alethiatoh,@jasontancy thanks!,"[jasont, thank]"
19996,4,2193527841,Tue Jun 16 08:34:46 PDT 2009,NO_QUERY,astroross,@DailyStatsmobi it was an amazing race and the...,"[dailystatsmobi, it, wa, an, amaz, race, and, ..."
19997,4,2193552007,Tue Jun 16 08:36:44 PDT 2009,NO_QUERY,Bellaleyla,"@Sky_1995 sorryyy. i gotta go now, my mom want...","[sky_, sorryyi, i, got, ta, go, now, my, mom, ..."
19998,4,2193575834,Tue Jun 16 08:38:42 PDT 2009,NO_QUERY,arikabu,@planetchuckie haha! i wonder why i made her m...,"[planetchucki, haha, i, wonder, whi, i, made, ..."


In [11]:
def remove_stopwords(l):
    """
    Remove English stopwords from a list of strings.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after stop words are removed.
    """
    stop_words = list(stopwords.words('english'))
    
    new_list = [word for word in l if word not in stop_words]
    return new_list

In [12]:
df['text_processed'] = df['text_processed'].apply(remove_stopwords)
df

,target,id,date,flag,user,text,text_processed
0,0,1467834053,Mon Apr 06 22:25:52 PDT 2009,NO_QUERY,thelazyboy,sleep soon... i just hate saying bye and see y...,"[sleep, soon, hate, say, bye, see, tomorrow, n..."
1,0,1467853356,Mon Apr 06 22:30:54 PDT 2009,NO_QUERY,dbmendel,Picked Mich St to win it all from the get go. ...,"[pick, mich, st, win, get, go, wa, feel, prett..."
2,0,1467873004,Mon Apr 06 22:36:03 PDT 2009,NO_QUERY,omgitsjo,I miss him. Can't wait to celebrate the Tar H...,"[miss, wait, celebr, tar, heel, win, thi, week..."
3,0,1467894600,Mon Apr 06 22:41:51 PDT 2009,NO_QUERY,dreaaa,throat is closing up and i had some string che...,"[throat, close, string, chees, good, idea]"
4,0,1467913111,Mon Apr 06 22:46:57 PDT 2009,NO_QUERY,nssmom,#3 woke up and was having an accident - &quot;...,"[woke, wa, accid, push, push, wa, cri, becaus,..."
...,...,...,...,...,...,...,...
19995,4,2193503437,Tue Jun 16 08:32:47 PDT 2009,NO_QUERY,alethiatoh,@jasontancy thanks!,"[jasont, thank]"
19996,4,2193527841,Tue Jun 16 08:34:46 PDT 2009,NO_QUERY,astroross,@DailyStatsmobi it was an amazing race and the...,"[dailystatsmobi, wa, amaz, race, think, want, ..."
19997,4,2193552007,Tue Jun 16 08:36:44 PDT 2009,NO_QUERY,Bellaleyla,"@Sky_1995 sorryyy. i gotta go now, my mom want...","[sky_, sorryyi, got, ta, go, mom, want, go, th..."
19998,4,2193575834,Tue Jun 16 08:38:42 PDT 2009,NO_QUERY,arikabu,@planetchuckie haha! i wonder why i made her m...,"[planetchucki, haha, wonder, whi, made, pic, h..."


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [13]:
bag_words =  nltk.FreqDist(df['text_processed'].explode().to_list())
bag_words

FreqDist({'go': 1720, 'get': 1411, 'day': 1347, 'wa': 1315, 'thi': 1228, 'good': 1154, 'love': 1106, 'work': 1080, 'like': 1003, 'u': 887, ...})

In [14]:
bag_words = sorted(bag_words.items(), key=lambda x:x[1], reverse=True)[:5000]
print(bag_words)

[('go', 1720), ('get', 1411), ('day', 1347), ('wa', 1315), ('thi', 1228), ('good', 1154), ('love', 1106), ('work', 1080), ('like', 1003), ('u', 887), ('got', 879), ('today', 853), ('time', 803), ('miss', 767), ('lol', 760), ('thank', 743), ('back', 738), ('want', 725), ('one', 713), ('know', 696), ('see', 642), ('im', 639), ('feel', 619), ('night', 600), ('realli', 582), ('think', 581), ('hope', 575), ('na', 571), ('still', 558), ('need', 549), ('well', 547), ('make', 536), ('watch', 525), ('new', 517), ('oh', 513), ('ha', 500), ('come', 490), ('much', 489), ('look', 484), ('last', 474), ('home', 465), ('sleep', 433), ('wish', 431), ('tomorrow', 428), ('haha', 428), ('morn', 420), ('twitter', 405), ('great', 389), ('whi', 377), ('week', 372), ('wait', 360), ('onli', 358), ('tri', 357), ('sad', 356), ('follow', 355), ('thing', 354), ('happi', 353), ('bad', 343), ('take', 341), ('fun', 328), ('veri', 327), ('gon', 326), ('would', 324), ('right', 323), ('tonight', 320), ('friend', 318), (

In [15]:
bag_list = []
for w in range(0,len(bag_words)):
    bag_list.append(bag_words[w][0])

bag_list

['go',
 'get',
 'day',
 'wa',
 'thi',
 'good',
 'love',
 'work',
 'like',
 'u',
 'got',
 'today',
 'time',
 'miss',
 'lol',
 'thank',
 'back',
 'want',
 'one',
 'know',
 'see',
 'im',
 'feel',
 'night',
 'realli',
 'think',
 'hope',
 'na',
 'still',
 'need',
 'well',
 'make',
 'watch',
 'new',
 'oh',
 'ha',
 'come',
 'much',
 'look',
 'last',
 'home',
 'sleep',
 'wish',
 'tomorrow',
 'haha',
 'morn',
 'twitter',
 'great',
 'whi',
 'week',
 'wait',
 'onli',
 'tri',
 'sad',
 'follow',
 'thing',
 'happi',
 'bad',
 'take',
 'fun',
 'veri',
 'gon',
 'would',
 'right',
 'tonight',
 'friend',
 'though',
 'sorri',
 'way',
 'hi',
 'nice',
 'say',
 'hate',
 'hour',
 'yeah',
 'weekend',
 'better',
 'could',
 'school',
 'show',
 'start',
 'peopl',
 'even',
 'awesom',
 'tweet',
 'soon',
 'play',
 'bed',
 'final',
 'ye',
 'never',
 'wan',
 'next',
 'ok',
 'use',
 'guy',
 'movi',
 'rain',
 'let',
 'dont',
 'sure',
 'first',
 'call',
 'sick',
 'plea',
 'year',
 'talk',
 'tire',
 'hey',
 'littl',
 'lon

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [16]:
def find_features(df):
    features = {}
    for w in bag_list:
        features[w] = (w in df)
    return features

In [17]:
feature_set = [(find_features(df), bool(target)) for (text, target) in zip(df['text_processed'], df['target'])]

In [18]:
feature_set[99]

({'go': False,
  'get': False,
  'day': False,
  'wa': False,
  'thi': False,
  'good': False,
  'love': False,
  'work': False,
  'like': False,
  'u': False,
  'got': False,
  'today': False,
  'time': False,
  'miss': False,
  'lol': False,
  'thank': False,
  'back': False,
  'want': False,
  'one': False,
  'know': False,
  'see': False,
  'im': False,
  'feel': False,
  'night': False,
  'realli': False,
  'think': False,
  'hope': False,
  'na': False,
  'still': False,
  'need': False,
  'well': False,
  'make': False,
  'watch': False,
  'new': False,
  'oh': False,
  'ha': False,
  'come': False,
  'much': False,
  'look': False,
  'last': False,
  'home': False,
  'sleep': False,
  'wish': False,
  'tomorrow': False,
  'haha': False,
  'morn': False,
  'twitter': False,
  'great': False,
  'whi': False,
  'week': False,
  'wait': False,
  'onli': False,
  'tri': False,
  'sad': False,
  'follow': False,
  'thing': False,
  'happi': False,
  'bad': False,
  'take': False,
  '

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [19]:
training_set = feature_set[:1000]
testing_set = feature_set[5000:]

In [20]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [21]:
classifier.show_most_informative_features(15) #I don't know why I am having this error because it should be nor any '<' sign neither float types

Most Informative Features


TypeError: ignored

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [22]:
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)

Classifier accuracy percent: 33.33333333333333


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here